# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [84]:
import pandas as pd


# Notebook Presentation

In [85]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [86]:
df = pd.read_csv('googleplaystore.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [87]:
df.sample(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
358,Firefox Focus: The privacy browser,COMMUNICATION,4.40,36880,4.0M,"1,000,000+",Free,0,Everyone,Communication,"July 6, 2018",5.2,5.0 and up
10074,Virtual Dice EX,GAME,4.30,40,3.6M,"5,000+",Free,0,Everyone,Board,"March 15, 2016",2.8,3.0 and up
4520,Q Call,TOOLS,NaN,20,16M,"1,000+",Free,0,Everyone,Tools,"July 16, 2018",2.100,4.0 and up
5552,Super AR,FAMILY,3.80,410,28M,"10,000+",Free,0,Teen,Entertainment,"December 18, 2017",2.1,4.0 and up
5624,Five Tries At Love Dating Sim,FAMILY,4.20,37165,22M,"1,000,000+",Free,0,Teen,Simulation,"February 14, 2018",5.1.0,2.3 and up


In [88]:
df.shape

(10841, 13)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns.

In [90]:
df.drop(columns=['Last Updated', 'Android Ver'], inplace=True)

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows.

In [91]:
df['Rating'].isnull().sum()

1474

In [92]:
df.dropna(inplace=True)
df.shape

(9362, 11)

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`.


In [93]:
df.duplicated().sum()

476

In [94]:
df.drop_duplicates(inplace=True)

If we do this without specifying how to identify duplicates, we see that 3 copies of Instagram are retained because they have a different number of reviews. We need to provide the column names that should be used in the comparison to identify duplicates. For example:

In [95]:
df[df['App'] == 'Instagram']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Current Ver
2545,Instagram,SOCIAL,4.50,66577313,Varies with device,"1,000,000,000+",Free,0,Teen,Social,Varies with device
2604,Instagram,SOCIAL,4.50,66577446,Varies with device,"1,000,000,000+",Free,0,Teen,Social,Varies with device
3909,Instagram,SOCIAL,4.50,66509917,Varies with device,"1,000,000,000+",Free,0,Teen,Social,Varies with device


In [96]:
df.drop_duplicates(subset=['App', 'Type', 'Price'], inplace=True)

In [97]:
df.shape

(8194, 11)

In [98]:
df['Size'] = df['Size'].replace(r'[KM]+$', '', regex=True)

In [99]:
df['Size'] = pd.to_numeric(df['Size'], errors='coerce')

In [100]:
average_size_by_category = df[df['Size'].notna()].groupby('Category')['Size'].mean()

def impute_size(row):
    if pd.isna(row['Size']):
        return average_size_by_category.get(row['Category'], 0)
    return row['Size']

df['Size'] = df.apply(impute_size, axis=1)

In [101]:
df['Installs'] = df['Installs'].str.replace(',', '').str.replace('+', '').astype(int)

<ipython-input-101-9affa6ed14c9>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Installs'] = df['Installs'].str.replace(',', '').str.replace('+', '').astype(int)


In [102]:
df.Reviews = df.Reviews.astype('int')

In [103]:
df.Price = df.Price.str.replace('$', '').astype(float)

<ipython-input-103-1a72ac3cc7c3>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.Price = df.Price.str.replace('$', '').astype(float)


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8194 entries, 0 to 10840
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             8194 non-null   object 
 1   Category        8194 non-null   object 
 2   Rating          8194 non-null   float64
 3   Reviews         8194 non-null   int64  
 4   Size            8194 non-null   float64
 5   Installs        8194 non-null   int64  
 6   Type            8194 non-null   object 
 7   Price           8194 non-null   float64
 8   Content Rating  8194 non-null   object 
 9   Genres          8194 non-null   object 
 10  Current Ver     8194 non-null   object 
dtypes: float64(3), int64(2), object(6)
memory usage: 768.2+ KB


What else should I know about the data?



So we can see that 13 different features were originally scraped from the Google Play Store.



Obviously, the data is just a sample out of all the Android apps. It doesn't include all Android apps of which there are millions.

I’ll assume that the sample is representative of the App Store as a whole. This is not necessarily the case as, during the web scraping process, this sample was served up based on geographical location and user behaviour of the person who scraped it - in our case Lavanya Gupta.

The data was compiled around 2017/2018. The pricing data reflect the price in USD Dollars at the time of scraping. (developers can offer promotions and change their app’s pricing).

I’ve converted the app’s size to a floating-point number in MBs. If data was missing, it has been replaced by the average size for that category.

The installs are not the exact number of installs. If an app has 245,239 installs then Google will simply report an order of magnitude like 100,000+. I’ve removed the '+' and we’ll assume the exact number of installs in that column for simplicity.

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [105]:
df.sort_values('Rating', ascending=False)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Current Ver
8327,The Divine Feminine App: the DF App,LIFESTYLE,5.00,8,6.70,1000,Free,0.00,Everyone,Lifestyle,0.0.4
6807,Jabbla BT,TOOLS,5.00,3,9.30,100,Free,0.00,Everyone,Tools,1.0
9194,UP EB Bill Payment & Details,SOCIAL,5.00,3,2.40,50,Free,0.00,Teen,Social,2.0
9218,EB Cash Collections,BUSINESS,5.00,1,4.30,5,Free,0.00,Everyone,Business,1.10
7270,CF Townsville,HEALTH_AND_FITNESS,5.00,4,14.00,100,Free,0.00,Everyone,Health & Fitness,4.2.2
...,...,...,...,...,...,...,...,...,...,...,...
4127,Speech Therapy: F,FAMILY,1.00,1,16.00,10,Paid,2.99,Everyone,Education,1.0
8820,DS Creator 2.0,TOOLS,1.00,2,4.40,500,Free,0.00,Everyone,Tools,2.0.180226.1
8875,DT future1 cam,TOOLS,1.00,1,24.00,50,Free,0.00,Everyone,Tools,3.1
5151,Clarksburg AH,MEDICAL,1.00,1,28.00,50,Free,0.00,Everyone,Medical,300000.0.81


The main drawback of this approach is that, apps with very few good ratings will also come at the top, leading to bias in our analysis. A better approach would be to consider only those app which are installed by more than a certain threshold and then sort by rating.

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please?

In [106]:
df.sort_values('Size', ascending=False).head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Current Ver
9170,Stickman Legends: Shadow Wars,GAME,4.40,38419,100.00,1000000,Paid,0.99,Everyone 10+,Action,2.3.24
4690,Vi Trainer,HEALTH_AND_FITNESS,3.60,124,100.00,5000,Free,0.00,Everyone,Health & Fitness,2.0.0.11
5862,Miami crime simulator,GAME,4.00,254518,100.00,10000000,Free,0.00,Mature 17+,Action,2.0
8409,Car Crash III Beam DH Real Damage Simulator 2018,GAME,3.60,151,100.00,10000,Free,0.00,Everyone,Racing,1.03
1793,Mini Golf King - Multiplayer Game,GAME,4.50,531458,100.00,5000000,Free,0.00,Everyone,Sports,3.04.1


Here we can clearly see that there seems to be an upper bound of 100 MB for the size of an app. A quick google search would also have revealed that this limit is imposed by the Google Play Store itself. It’s interesting to see that a number of apps actually hit that limit exactly.

# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [108]:
df.sort_values('Reviews', ascending=False).head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Current Ver
2544,Facebook,SOCIAL,4.10,78158306,17.53,1000000000,Free,0.00,Teen,Social,Varies with device
336,WhatsApp Messenger,COMMUNICATION,4.40,69119316,13.64,1000000000,Free,0.00,Everyone,Communication,Varies with device
2545,Instagram,SOCIAL,4.50,66577313,17.53,1000000000,Free,0.00,Teen,Social,Varies with device
335,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,13.64,1000000000,Free,0.00,Everyone,Communication,Varies with device
1670,Clash of Clans,GAME,4.60,44891723,98.00,100000000,Free,0.00,Everyone 10+,Strategy,10.322.16


If you look at the number of reviews, you can find the most popular apps on the Android App Store. These include the usual suspects: Facebook, WhatsApp, Instagram etc. What’s also notable is that the list of the top 50 most reviewed apps does not include a single paid app!

# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [112]:
ratings = df['Content Rating'].value_counts()
ratings

Everyone           6616
Teen                912
Mature 17+          357
Everyone 10+        305
Adults only 18+       3
Unrated               1
Name: Content Rating, dtype: int64

In [113]:
import plotly.express as px

In [114]:
fig = px.pie(labels=ratings.index,
values=ratings.values,
title="Content Rating",
names=ratings.index,
)
fig.update_traces(textposition='outside', textinfo='percent+label')

fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



To create a donut 🍩 chart, we can simply add a value for the hole argument:

In [115]:
fig = px.pie(labels=ratings.index,
values=ratings.values,
title="Content Rating",
names=ratings.index,
hole=0.6,
)
fig.update_traces(textposition='inside', textfont_size=15, textinfo='percent')

fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install?

Check the datatype of the Installs column.

Count the number of apps at each level of installations.

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first.

In [116]:
df[['App', 'Installs']].groupby('Installs').count()

,App
Installs,
1,3
5,9
10,69
50,56
100,303
500,199
1000,697
5000,424
10000,987


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [118]:
df.sort_values('Price', ascending=False).head(20)[['App', 'Price']]

,App,Price
4367,I'm Rich - Trump Edition,400.00
4197,most expensive app (H),399.99
5354,I am Rich Plus,399.99
5359,I am rich(premium),399.99
5362,I Am Rich Pro,399.99
4362,💎 I'm rich,399.99
5358,I am Rich!,399.99
5356,I Am Rich Premium,399.99
5373,I AM RICH PRO PLUS,399.99
5364,I am rich (Most expensive app),399.99


What’s going on here? There are 15 I am Rich Apps in the Google Play Store apparently. They all cost \$300.00 or more, which is the main point of the app. The story goes that in 2008, Armin Heinrich released the very first I am Rich app in the iOS App Store for $999.90. The app does absolutely nothing. It just displays the picture of a gemstone and can be used to prove to your friends how rich you are. Armin actually made a total of 7 sales before the app was hastily removed by Apple. Nonetheless, it inspired a bunch of copycats on the Android App Store, but if you search today, you’ll find all of these apps have disappeared as well. The high installation numbers are likely gamed by making the app was available for free at some point to get reviews and appear more legitimate.

### The most expensive apps sub $250

In [120]:
df = df[df['Price'] < 250]
df.sort_values('Price', ascending=False).head(20)[['App', 'Price']]

,App,Price
2253,Vargo Anesthesia Mega App,79.99
2414,LTC AS Legal,39.99
5360,I am Rich Person,37.99
2301,A Manual of Acupuncture,33.99
2266,EMT PASS,29.99
5489,AP Art History Flashcards,29.99
3039,Golfshot Plus: Golf GPS,29.99
2298,PTA Content Master,29.99
2251,Human Anatomy Atlas 2018: Complete 3D Human Body,24.99
2277,"Muscle Premium - Human Anatomy, Kinesiology, B...",24.99


### Highest Grossing Paid Apps (ballpark estimate)

In [121]:
df['Revenue Estimate'] = df['Price'] * df['Installs']
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Current Ver,Revenue Estimate
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.10,159,19.00,10000,Free,0.00,Everyone,Art & Design,1.0.0,0.00
1,Coloring book moana,ART_AND_DESIGN,3.90,967,14.00,500000,Free,0.00,Everyone,Art & Design;Pretend Play,2.0.0,0.00
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.70,87510,8.70,5000000,Free,0.00,Everyone,Art & Design,1.2.4,0.00
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.50,215644,25.00,50000000,Free,0.00,Teen,Art & Design,Varies with device,0.00
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.30,967,2.80,100000,Free,0.00,Everyone,Art & Design;Creativity,1.1,0.00


In [124]:
df.sort_values('Revenue Estimate', ascending=False).head(10)[['App', 'Category', 'Price', 'Installs', 'Revenue Estimate']]

,App,Category,Price,Installs,Revenue Estimate
2241,Minecraft,FAMILY,6.99,10000000,"69,900,000.00"
4034,Hitman Sniper,GAME,0.99,10000000,"9,900,000.00"
7417,Grand Theft Auto: San Andreas,GAME,6.99,1000000,"6,990,000.00"
2883,Facetune - For Free,PHOTOGRAPHY,5.99,1000000,"5,990,000.00"
5578,Sleep as Android Unlock,LIFESTYLE,5.99,1000000,"5,990,000.00"
8804,DraStic DS Emulator,GAME,4.99,1000000,"4,990,000.00"
4133,Weather Live,WEATHER,5.99,500000,"2,995,000.00"
2242,Card Wars - Adventure Time,FAMILY,2.99,1000000,"2,990,000.00"
5631,Five Nights at Freddy's,GAME,2.99,1000000,"2,990,000.00"
8860,Bloons TD 5,FAMILY,2.99,1000000,"2,990,000.00"


The top spot of the highest-grossing paid app goes to … Minecraft at close to $70 million. It’s quite interesting that Minecraft (along with Bloons and Card Wars) is actually listed in the Family category rather than in the Game category. If we include these titles, we see that 4 out the top 10 highest-grossing apps are games. The Google Play Store seems to be quite flexible with its category labels.

# Plotly Bar Charts & Scatter Plots: Analysing App Categories

### Vertical Bar Chart - Highest Competition (Number of Apps)

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

### Category Concentration - Downloads vs. Competition

**Challenge**:
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this.

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log')

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html).


# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data?

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/).

* Find a way to set the colour scale using the color_continuous_scale parameter.
* Find a way to make the color axis disappear by using coloraxis_showscale.

# Grouped Bar Charts: Free vs. Paid Apps per Category

**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart:

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category.

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value).

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart.

<img src=https://imgur.com/uVsECT3.png>


# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below:

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories.

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.